# TensorFlow Regression Example

## Imports

In [ ]:


# remember TensorFlow and SciKit-Learn up here

## Creating Data

#### One Million Points!

#### Noise

#### Now, for the data

# TensorFlow

## Batch Size

> We will take the data in batches (1 000 000 points is a lot to pass in at once).

#### Variables

#### Placeholders

#### Graph

#### Loss Function

#### Optimizer

#### Initialize Variables

### Session

### Results

## tf.estimator API

> Much simpler API for basic tasks like regression! We'll talk about more abstractions like TF-Slim later on.

### Train Test Split

> We haven't actually performed a train test split yet! So let's do that on our data now and perform a more realistic version of a Regression Task

### Set up Estimator Inputs

### Train the Estimator

### Evaluation

### Predictions

_That's all for now!_